In [ ]:
# pip uninstall tensorflow


Found existing installation: tensorflow 2.20.0
Uninstalling tensorflow-2.20.0:
  Would remove:
    /home/muki/CSP/TensorTetris/.venv/bin/import_pb_to_tensorboard
    /home/muki/CSP/TensorTetris/.venv/bin/saved_model_cli
    /home/muki/CSP/TensorTetris/.venv/bin/tensorboard
    /home/muki/CSP/TensorTetris/.venv/bin/tf_upgrade_v2
    /home/muki/CSP/TensorTetris/.venv/bin/tflite_convert
    /home/muki/CSP/TensorTetris/.venv/bin/toco
    /home/muki/CSP/TensorTetris/.venv/lib/python3.11/site-packages/tensorflow-2.20.0.dist-info/*
    /home/muki/CSP/TensorTetris/.venv/lib/python3.11/site-packages/tensorflow/*
Proceed (Y/n)? 

In [2]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
# pyright: ignore[reportMissingImports]
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.networks import network

from tf_agents.policies import py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import scripted_py_policy

from tf_agents.policies import tf_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import actor_policy
from tf_agents.policies import q_policy
from tf_agents.policies import greedy_policy

from tf_agents.trajectories import time_step as ts

2026-01-27 17:04:09.624369: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-27 17:04:09.666218: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-27 17:04:11.394900: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-27 17:04:20.909472: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [ ]:
"""
Personal Tetris Implementaiton
"""

#TODO: maybe change from 2d list to numpy?

# import numpy as np
import random
import sys
import copy

class Tetris:

    # Tetris game class
    config = {
        "rows": 14,
        "cols": 6, # Min 6
        "render": False,
        "screen_width": 300, # Pygame screen 
        "screen_height": 700
    }

    # All pieces at all rotations
    tetris_pieces = {
        0: { # I
            0: [(0,0), (1,0), (2,0), (3,0)],
            90: [(1,0), (1,1), (1,2), (1,3)],
            180: [(3,0), (2,0), (1,0), (0,0)],
            270: [(1,3), (1,2), (1,1), (1,0)],
        },
        1: { # T
            0: [(1,0), (0,1), (1,1), (2,1)],
            90: [(0,1), (1,2), (1,1), (1,0)],
            180: [(1,2), (2,1), (1,1), (0,1)],
            270: [(2,1), (1,0), (1,1), (1,2)],
        },
        2: { # L
            0: [(1,0), (1,1), (1,2), (2,2)],
            90: [(0,1), (1,1), (2,1), (2,0)],
            180: [(1,2), (1,1), (1,0), (0,0)],
            270: [(2,1), (1,1), (0,1), (0,2)],
        },
        3: { # J
            0: [(1,0), (1,1), (1,2), (0,2)],
            90: [(0,1), (1,1), (2,1), (2,2)],
            180: [(1,2), (1,1), (1,0), (2,0)],
            270: [(2,1), (1,1), (0,1), (0,0)],
        },
        4: { # Z
            0: [(0,0), (1,0), (1,1), (2,1)],
            90: [(0,2), (0,1), (1,1), (1,0)],
            180: [(2,1), (1,1), (1,0), (0,0)],
            270: [(1,0), (1,1), (0,1), (0,2)],
        },
        5: { # S
            0: [(2,0), (1,0), (1,1), (0,1)],
            90: [(0,0), (0,1), (1,1), (1,2)],
            180: [(0,1), (1,1), (1,0), (2,0)],
            270: [(1,2), (1,1), (0,1), (0,0)],
        },
        6: { # O/Square
            0: [(1,0), (2,0), (1,1), (2,1)],
            90: [(1,0), (2,0), (1,1), (2,1)],
            180: [(1,0), (2,0), (1,1), (2,1)],
            270: [(1,0), (2,0), (1,1), (2,1)],
        }
    }
    
    def __init__(self, _render):
        self.config["render"] = _render
        self.reset()

    def reset(self):
        # clear and setup board

        # BOARD SETUP
        # 0 = Empty space
        # 1 = Piece
        self.cols = self.config["cols"]
        self.rows = self.config["rows"]
        self.board = [[0] * self.cols  for _ in range(self.rows)]
        self.score = 0
        self.current_piece = 0
        self.next_piece = random.randint(0,6)
        self.pos = [0, int((self.config["cols"] / 2) - 2)] # Middle of top of board
        self.rotation = 0
        
        return self.board

    # Clears all lines that are full
    def _clear_lines(self):
        # Check for full lines, clear if needed
        lines_full = []
        
        # Get full lines
        for i, row in board:
            if 0 not in row:
                lines_full.append(i)
        
        # Create new board without full lines
        board = [row for index, row in enumerate(board) if index not in lines_full]
        
        # Insert new lines at top
        for line in lines_full:
            board.insert(0, [0 for _ in range(self.cols)])

        # Give score based on lines cleared
        if (len(lines_full) == 4): #TETRIS (extra points)
            score += 8
        else:
            score += len(lines_full)

    # Check for collisions with pieces on board or side of board
    def _is_colliding(self):
        piece = self.tetris_pieces[self.current_piece][self.rotation]

        for x, y in piece:
            x += self.pos[0]
            y += self.pos[1]

            if x not in range(self.cols) \
            or y not in range(self.rows) \
            or self.board[y][x] == 1:
                return True
        return False 
            
    # Adds current piece to board, randomly select another piece, end game if colliding
    def _add_piece(self):
        piece = self.tetris_pieces[self.current_piece][self.rotation]

        for x, y in piece:
            x += self.pos[0]
            y += self.pos[1]

            self.board[y][x] = 1

        # add new piece to board and check for collisions
        self.current_piece = self.next_piece
        self.next_piece = random.randint(0,6)

        self.pos = [0, int((self.cols / 2) - 2)]
        self.rotation = 0

        if self._is_colliding():
            # Game over
            self._game_over()


    def _game_over(self):
        #TODO: Figure out scoring system
        # Theoretical scoring/reward system
        # 4 line clear 8 points
        # else lines cleared = points (3 lines = 3, etc)
        # death = -10 reward?

        #with open("scores.txt", "w") as f:
        #    f.write(self.score)  
        #sys.exit()
        self.reset()

    # Angle is 90 or -90
    def _rotate(self, angle):
        self.rotation += angle

        if self.rotation == 360:
            self.rotation = 0
        elif self.rotation < 0:
            self.rotation = 270

    def step(self, action):
        # Do given action to board
        if (action == "COUNTERCLOCKWISE"):
            self._rotate(-90)
        elif(action == "CLOCKWISE"):
            self._rotate(90)
        elif(action == "RIGHT"):
            self.pos[0] += 1
        elif(action == "LEFT"):
            self.pos[0] -= 1

        # Check collisions and reverse action if not allowed
        if self._is_colliding():
            print("INVALID MOVE")
            # Reverse action
            if (action == "COUNTERCLOCKWISE"):
                self._rotate(90)
            elif(action == "CLOCKWISE"):
                self._rotate(-90)
            elif(action == "RIGHT"):
                self.pos[0] -= 1
            elif(action == "LEFT"):
                self.pos[0] += 1
        
        # Drop piece down by one and check collision, add to board if colliding
        self.pos[1] += 1
        if self._is_colliding():
            print("PIECE PLACED")
            self.pos[1] -= 1
            self._add_piece()

        if (self.config["render"]):
            self._render()

        #TODO: Figure out how to return board, current piece, rotation, and next piece in one input tensor
        # maybe int tensor? (straighten out 2d list into 1d array by row and add other info at end)
        return self.board 
    
    # Creates deepcopy of current board state with current piece
    def get_board_copy(self):
        tempboard = copy.deepcopy(self.board)

        for x, y in self.tetris_pieces[self.current_piece][self.rotation]:
            x += self.pos[0]
            y += self.pos[1]

            tempboard[y][x] = 2
        
        return tempboard

    def _render(self):
        # Add current piece to temp board
        tempboard = copy.deepcopy(self.board)

        for x, y in self.tetris_pieces[self.current_piece][self.rotation]:
            x += self.pos[0]
            y += self.pos[1]

            tempboard[y][x] = 1

        for row in tempboard:
            print(row)
        print("")
        


In [ ]:
# Random agent implemented to display with Pygame

import pygame

# Action_spec for 6 possible actions
# RIGHT LEFT COUNTERCLOCKWISE CLOCKWISE NOOP
action_spec = array_spec.BoundedArraySpec(
    shape=(),
    dtype=np.int32,
    minimum=0,
    maximum=4,
    name='action'
)

# Setup tf for random actions
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)

# Setup Pygame
RED = (255, 0, 0)
BLACK = (0, 0, 0)

tetris = Tetris(False)
pygame.init()
screen = pygame.display.set_mode((300, 700))
pygame.event.clear()
running = True

while running:

    # Wait for pygame event
    event = pygame.event.wait()

    # Only respond to key presses
    if event.type == pygame.quit:
        running = False
        pygame.quit()
        sys.exit()
    elif event.type == pygame.KEYDOWN:
        if event.key == pygame.K_ESCAPE:
            running = False
            pygame.quit()
            sys.exit()
        else:

            # Get action from random agent and apply
            action = my_random_tf_policy.action(time_step)
            match action[0]:
                case 0:
                    print("RIGHT")
                    tetris.step("RIGHT")
                case 1:
                    print("LEFT")
                    tetris.step("LEFT")
                case 2:
                    print("CC")
                    tetris.step("COUNTERCLOCKWISE")
                case 3:
                    print("C")
                    tetris.step("CLOCKWISE")
                case 4:
                    print("N")
                    tetris.step("NOOP")
            
            # Get board copy for rendering
            board = tetris.get_board_copy()
            
            # Draw screen
            screen.fill(BLACK)

            # 
            for row_index, row in enumerate(board):
                for col_index, tile_value in enumerate(row):
                    # Calculate the position for the current tile
                    x = col_index * 50
                    y = row_index * 50

                    if (tile_value) == 0:
                        color = BLACK
                    else:
                        color = RED

                    # Draw the rectangle
                    pygame.draw.rect(screen, color, (x, y, 50, 50))

            # Update the display
            pygame.display.flip()


C
Rotation: 0
N
Rotation: 90
LEFT
Rotation: 90
N
Rotation: 90
N
Rotation: 90
N
Rotation: 90
C
Rotation: 90
INVALID MOVE
LEFT
Rotation: 90
INVALID MOVE
RIGHT
Rotation: 90
N
Rotation: 90
PIECE PLACED
RIGHT
Rotation: 0
CC
Rotation: 0
RIGHT
Rotation: 270
RIGHT
Rotation: 270
CC
Rotation: 270
INVALID MOVE
C
Rotation: 270
INVALID MOVE
RIGHT
Rotation: 270
CC
Rotation: 270
INVALID MOVE
CC
Rotation: 270
INVALID MOVE
C
Rotation: 270
INVALID MOVE
PIECE PLACED
N
Rotation: 0
C
Rotation: 0
CC
Rotation: 90
C
Rotation: 0
RIGHT
Rotation: 90
N
Rotation: 90
CC
Rotation: 90
C
Rotation: 0
INVALID MOVE
CC
Rotation: 0
INVALID MOVE
PIECE PLACED
N
Rotation: 0
RIGHT
Rotation: 0
N
Rotation: 0
C
Rotation: 0
RIGHT
Rotation: 90
C
Rotation: 90
RIGHT
Rotation: 180
C
Rotation: 180
PIECE PLACED
CC
Rotation: 0
RIGHT
Rotation: 270
CC
Rotation: 270
CC
Rotation: 180
C
Rotation: 90
CC
Rotation: 180
PIECE PLACED
N
Rotation: 0
RIGHT
Rotation: 0
N
Rotation: 0
PIECE PLACED
C
Rotation: 0
N
Rotation: 90
N
Rotation: 90
PIECE PLACED

SystemExit: 

In [ ]:
# Tetris game with console rendering

tetris = Tetris(True)
userinput = ""

while (userinput != "STOP"):
    userinput = input()

    match userinput:
        case "r":
            print("RIGHT")
            tetris.step("RIGHT")
        case "l":
            print("LEFT")
            tetris.step("LEFT")
        case "cc":
            print("CC")
            tetris.step("COUNTERCLOCKWISE")
        case "c":
            print("C")
            tetris.step("CLOCKWISE")
        case "n":
            print("N")
            tetris.step("NOOP")

CC
Rotation: 0
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

CC
Rotation: 270
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

CC
Rotation: 180
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

CC
Rotation: 90
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0]
[